In [7]:
import torch
import matplotlib.pyplot as plt

from utils import Dataset, load_data, prepare_x, get_label, data_classification
from trainer import batch_gd, evaluate
from models import DeepLOB

In [3]:
batch_size = 32
k = 10
num_classes = 3
T = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# Load the data
data_path = './data/'
train, val, test = load_data(data_path)

dataset_train = Dataset(data=train, k=k, num_classes=num_classes, T=T)
dataset_val = Dataset(data=val, k=k, num_classes=num_classes, T=T)
dataset_test = Dataset(data=test, k=k, num_classes=num_classes, T=T)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape)

(149, 203800) (149, 50950) (149, 139587)


IndexError: index 10 is out of bounds for axis 1 with size 5

In [8]:
x = prepare_x(train)
y = get_label(train)
x, y = data_classification(x, y, T)

In [9]:
y

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 3., 1.],
       [2., 2., 1., 1., 1.],
       ...,
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.]])

In [ ]:
model = DeepLOB(y_len = dataset_train.num_classes)
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, eps=1.0)      # set as in the paper or use lr=0.0001

In [ ]:
model, train_losses, val_losses = batch_gd(model, criterion, optimizer, train_loader, val_loader, k = k, epochs=150)

plt.figure(figsize=(15,6))
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='validation loss')
plt.title('Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
report_k10 = evaluate(model, test_loader)